In [1]:
#!/usr/bin/env python
""" 
"""
__author__ = 'Adam Duster'
__copyright__ = ''
__credits__ = ['Adam Duster']
__license__ = 'CC-BY-SA'
__version__ = '0.1'
__email__ = 'adam.duster@ucdenver.edu'
__status__ = 'Development'

import argparse
def get_args(args=None):
    """ This is written as a default funtion to put at beginning of all Python
    scripts which require command line arguments. This uses the argparse module
    which must be declared in the main program to ensure that the object is able
    to be used by the caller
    --Adam Duster 21 June 2017
    """
    parser = argparse.ArgumentParser(description='see header of python script')
    parser.add_argument(
        '-i',
        '--if_paths',
        help='The first and second files in order',
        required=True)
    parser.add_argument(
        '-v',
        '--verbose',
        help='Controls the level of output, use multipe v for more output',
        required=False,
        action='count',
        default=0)
    parser.add_argument(
        '-s',
        '--step',
        help='The change in coordinates between the two files',
        required=False,
        default=1e-6,
        type=float
    )
    parser.add_argument(
        '-d',
        '--debug',
        help='Enter debug mode',
        required=False,
        action='store_true',
        default=False)
    return parser.parse_args(args)

## Vars
arg_vals = None
args = get_args(['-i', 'oh-bpsf.h5', '-s', '1e-6'])



In [2]:
# Calculate the numerical gradient
import h5py as h5
import numpy as np

#key = 'o_rad_cartesian_gradient'
#key = 'h_radial_sym_funcs'
key = 'o_radial_sym_funcs'
ckey = 'o_rad_cartesian_gradient'
check = False
check = True
print(args.if_paths[0])
print(args.if_paths[1])
k = 0
with h5.File(args.if_paths, 'r') as ifi:
    delta_symfuncs = (ifi[key][1,:] - ifi[key][0,:] ) / 1e-6
    print(ckey, 'shape: ', ifi[ckey].shape)
    ograd = ifi[ckey][1]
    hgrad = ifi['h_rad_cartesian_gradient'][1]
    o_symfuncs = ifi['o_radial_sym_funcs'][1]
    h_symfuncs = ifi['h_radial_sym_funcs'][1]
#print(calc)

/home/aduster/anaconda3/envs/mda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


o
h
o_rad_cartesian_gradient shape:  (2, 48, 151, 3)


In [3]:
for i in range(delta_symfuncs.size):
    if delta_symfuncs[i] == 0 and calc[i] == 0:
        continue
    print(delta_symfuncs[i], calc[i], calc[i]-delta_symfuncs[i], (calc[i]-delta_symfuncs[i])/delta_symfuncs[i])

NameError: name 'calc' is not defined

In [4]:
npa = np.asarray


xi = npa([0.,0.,0.])
xj = npa([1.,0.,0.])
def calc_gi(xi, xj, rs=0.800, eta=19.531, rc=8.0, verbose=False):
    vij = xj - xi
    rij = np.linalg.norm(vij)
    drijdi = -vij/rij
    drijdj = drijdi
    fc = 0.5 * (np.cos(np.pi*rij/rc) + 1)
    gauss = np.exp(-eta*(rij - rs)**2)
    gi = gauss*fc
    if verbose:
        print("rij, then rs", rij, rs)
        print("(rij-rs)**2",(rij-rs)**2)
        print("myexp", gauss)
        print("myfc", fc)
    
    return gi

cgi = calc_gi(xi, xj)
print("calc gi", cgi)
print("reference_gi", o_symfuncs[0])


calc gi 0.44041252110981277
reference_gi 0.44040811700662225


In [5]:
# Lets calculate the O symfunc numerical gradient
s = 1e-6
xi = npa([0.,0.,0.])
xj = npa([1.,0.,0.])
xib = np.copy(xi)
xib = xib[0] + s

agi = calc_gi(xi, xj)
bgi = calc_gi(xib, xj)
print("numerical gradient: ", (bgi - agi)/s)
print(ograd[0,0])


numerical gradient:  3.475082208603819
[3.47508988 0.         0.        ]


In [6]:
# Lets calculate the H symfunc numerical gradient
s = 1e-6
xi = npa([1.,0.,0.])
xj = npa([0.,0.,0.])
xib = np.copy(xi)
xib[0] = xib[0] + s

agi = calc_gi(xi, xj)
bgi = calc_gi(xib, xj)
print(np.linalg.norm(xib-xj))
print("calculated symfunc:", agi)
print("reference symfunc:", h_symfuncs[24])
print("numerical gradient: ", (bgi - agi)/s)
print(hgrad[24,0])


1.000001
calculated symfunc: 0.44041252110981277
reference symfunc: 0.44040811700662225
numerical gradient:  -3.4750755017465274
[3.47508988 0.         0.        ]


In [17]:
import h5py as h5
import numpy as np
hrcg = 'h_rad_cartesian_gradient'
orcg = 'o_rad_cartesian_gradient'
hacg = 'h_ang_cartesian_gradient'
oacg = 'p_ang_cartesian_gradient'

with h5.File('oh-bpsf.h5') as ifi:
    gs = ifi[hrcg][:]

In [20]:
print(gs[0,24,0,:])


[3.47510006 0.         0.        ]
